In [1]:

import numpy as np
from scipy.io import loadmat
from helper_functions import *


In [2]:
# def pmhmm(X, T, M=2, K=4, cyc=100, tol=0.0001, iters=10):

X = loadmat('X.mat')['X']
test = loadmat('test.mat')

M=2
K=2
cyc=100
tol=0.0001
iters=10

p = X.shape[1]
T = X.shape[0]

tiny = test['tiny']
Cov = test['Cov']
XX = test['XX']
Xalt = test['Xalt']

Pi = loadmat('vars.mat')['Pi']
P = loadmat('vars.mat')['P']
Mu = loadmat('vars.mat')['Mu']


LL = []
lik = 0
likbase = 0

k1 = test['k1'] #(dot(2, pi)) ** (- p / 2)
mf = np.ones((T,M*K)) / K

alpha = np.zeros((T,M*K))
beta = np.zeros((T,M*K))
logmf = np.log(mf)
h = np.ones((T,M*K)) / K
exph = np.exp(h)


In [2]:
for cycle in range(cyc):

    gamma = np.zeros((T,M*K))
    Eta = np.zeros((M*K,M*K))
    Xi = np.zeros((M*K, K))
    iCov = np.linalg.inv(Cov)
    k2 = k1 / np.sqrt(np.linalg.det(Cov))

    itermf = iters

    for l in range(iters):

        mfo = np.copy(mf)
        logmfo = np.copy(logmf)

        # compute h based on mf

        for i in range(T):
            d2 = i
            d = Xalt[d2, :]
            yhat = np.dot(mf[d2, :], Mu)
            for j in range(M):
                d1 = np.arange(j*K, j*K + K)
                Muj = Mu[d1, :]
                h[d2, d1] = Muj @ iCov @ (d-yhat).T + Muj @ iCov @ Muj.T @ mf[d2, d1].T - 0.5 * np.diag(Muj @ iCov @ Muj.T)
                h[d2, d1] = h[d2, d1] - max(h[d2, d1])*np.ones((1,K))
        exph = np.exp(h)

        # compute mf based on h using BW (EM)

        scale = np.zeros((T, M))

        for j in range(M):

            # alpha
            d1 = np.arange(j*K, j*K + K)
            d2 = 0
            alpha[d2, d1] = exph[d2,d1] * Pi[:,j].T
            scale[d2, j] = np.sum(alpha[d2,d1], axis=0) + tiny
            alpha[d2, d1] = alpha[d2,d1] / scale[d2,j]
            for i in range(1,T):
                d2 = i
                alpha[d2,d1] = alpha[d2-1, d1]@P[d1, :]*exph[d2,d1]
                scale[d2, j] = np.sum(alpha[d2,d1], axis=0) + tiny
                alpha[d2, d1] = alpha[d2,d1] / scale[d2,j]

            # beta
            d2=T-1
            beta[d2,d1] = np.ones((1, K)) / scale[d2, j]
            for i in range(T - 2, -1, -1):
                d2 = i
                beta[d2,d1] = beta[d2 + 1,d1] * exph[d2 + 1,d1] @ P[d1, :].T
                beta[d2,d1] = beta[d2,d1] / scale[d2, j]

            # mf
            mf[:, d1] = alpha[:,d1] * beta[:,d1]
            mf[:,d1] = np.divide(mf[:,d1], (np.ones((2, 1)) * np.sum(mf[:,d1], axis=1) + tiny).T)

        mf = np.where(mf<tiny, tiny, mf)
        logmf = np.log(mf)
        delmf = np.sum(mf*logmf)-np.sum(mf*logmfo)
        if delmf < T*1e-6:
            itermf = l
            break

    oldlik = lik
    lik = calc_mf_like(Xalt, T, mf, M, K, Mu, Cov, P, Pi)
    LL.append(lik)

    print('cycle: ', cycle,', mf iters: ', itermf,' loglike: ', lik)

    if cycle <= 1:
        likbase = lik
    elif lik-likbase < 1+tol * oldlik-likbase:
        print('cycle break', cycle)
        break


In [52]:
test = [0,1,2]
np.where(test<tiny, tiny, test)

array([[9.85967654e-305, 1.00000000e+000, 2.00000000e+000]])

In [32]:

test = np.array([int(x) for x in bin(1)[2:]])
if len(test) < K:
    zeros = np.zeros((K-len(test)))

In [6]:
# (np.ones((2, 1)) * np.sum(mf[:,d1], axis=1) + tiny).T

In [7]:
# np.allclose(iCov, test['iCov'])